In [ ]:
import ipywidgets
from IPython.display import display, HTML, clear_output
import qgrid
import pandas as pd
from ipywidgets import Layout, widgets
from ipywidgets.widgets import Text, Label, Box, HBox, VBox, Button, Textarea, Dropdown, SelectMultiple, RadioButtons
import ipyleaflet as leaf

In [ ]:
querylist = widgets.Dropdown(options=['Virginia Customer Data',
                                     'Ohio Customer Data',
                                     'Nevada Customer Data',
                                     'U.S. Virgin Islands Customer Data',
                                     'Idaho Customer Data',
                                     'Wisconsin Customer Data'], layout=Layout(width='auto', margin='10px 0px 20px 0px'))

refreshqueries = Button(description=' Refresh Datasets',
                  button_style='success',
                  icon='retweet',
                  tooltip='Click to refresh your list of available datasets', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 20px 5px'))

ql_label = widgets.HTML('<h4>Select a Dataset</h4>')

dxwf = widgets.Text(layout=Layout(width='auto', margin='20px 0px 10px 0px'), 
                    placeholder='*OR* input your data csv filename for upload', continuous_update = True)

entity = widgets.Text(layout=Layout(width='auto', margin='20px 0px 10px 0px'), 
                      placeholder='Add a name for your dataset', continuous_update = True)

In [ ]:
filteron_bb = widgets.Button(description=' Filter Table by Map Box(es)',
                  button_style='success',
                  icon='filter',
                  tooltip='Click to filter your data table by the bounding boxes currently drawn on the map',
                  layout=Layout(width='auto', height='40px'))

restore_table = Button(description=' Restore the Full Table',
                  button_style='success',
                  icon='retweet',
                  tooltip='Click to restore your full table of data',
                  layout=Layout(width='auto', height='40px'))

cm_name = widgets.Text(placeholder='Name for saving your filtered data set',
                       layout=Layout(width='auto', margin='20px 0px 10px 0px'),
                       continuous_update = True)

cm_save = Button(description=' Publish to Database',
                  button_style='success',
                  icon='save',
                  tooltip='Click to publish the data from your currently displayed table to the database',
                  layout=Layout(width='auto', height='40px'))

importdx = Button(description=' Import customer data',
                  button_style='success',
                  icon='download',
                  tooltip='Click to import your customer data',
                  layout=Layout(width='auto', height='40px'))

app_dataset = Button(description=' Append another dataset',
                  button_style='success',
                  icon='plus',
                  tooltip='Click to append another dataset',  
                  layout=Layout(width='auto', height='40px'))

save_hmlayer = Button(description=' Save Current Heatmap Layer(s)',
                  button_style='success',
                  icon='save',
                  tooltip='Click to save your current heatmap layer(s)',
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

save_loclayer = Button(description=' Save Current Locations Layer',
                  button_style='success',
                  icon='save',
                  tooltip='Click to save your current locations layer',
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

savelayer_name = widgets.Text(layout=Layout(width='auto', height = '40 px', margin='5px 5px 5px 5px'), placeholder = 'Name to save your layer as', continuous_update = True)


In [ ]:
plot_pictures = Button(description=' Plot geo-tagged photos',
                  button_style='success',
                  icon='camera',
                  tooltip='Click to plot geo-tagged photos', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

dtable_summary = """<style type="text/css">
.dsummary {background-color: #3C598E;    
        background-image: linear-gradient(#23761f, #79da74);    
        color: white;    
        text-shadow: 3px 3px 5px rgba(0,0,0,0.5); 
        text-align: left;
        font-size: 15px;
        padding: 15px; 
        }
</style>
<div class="dsummary">
<span><h4>Summary</h4></span><br>
Number of Events:56982/174385<br>
Total Days: 260/365<br>
Total Locs (by 8-char GH): 3425/6852<br>
First Event: 1 January 2019<br>
Last Event: 31 October 2019<br>
Data Source Types: Customer logs, pins<br>
</div>"""

summaryw = widgets.HTML(dtable_summary)

summaryw.layout=Layout(width='auto')

In [ ]:
filename_mastermap = Text(description='',
                          layout=Layout(width='auto'),
                          placeholder='Filename for your map output (.html will be added)',
                          continuous_update = True)

label_mastermap = widgets.HTML(
  value = '<b>Saving the Map with Current Layers</b>')

save_mastermap = Button(description=' Save Map',
                  button_style='success',
                  icon='save',
                  tooltip='Click to save your interactive map to your Jupyter homepage as an html file',  
                  layout=Layout(width='auto', height='40px'))

In [ ]:
basemaps = ["World Street Map",
            "Google Earth Imagery",
            "Stamen Watercolor",
            "Combined Maps"]

dml_label = widgets.HTML(
  value = '<b>Dual map Layers?</b>')

dml_w = widgets.Checkbox(value=False)

dml_left_label = widgets.HTML(value = '<b>Left Map Layer</b>')
dml_right_label = widgets.HTML(value = '<b>Right Map Layer</b>')
dml_left = widgets.Dropdown(options=basemaps)
dml_right = widgets.Dropdown(options=basemaps)

gh_info_label = widgets.HTML("<b>Perform a reverse geo lookup on a specific geohash</b>", layout=Layout(margin='10px 0px 10px 0px'))

gh_text = widgets.Text(layout=Layout(width='auto'), continuous_update=True, placeholder="Paste in a geohash for a reverse geo lookup on its center point")

gh_info = Button(description=' Reverse Geolookup',
                  button_style='success',
                  icon='location-arrow',
                  tooltip='Click to perform a reverse geo lookup on a specific geohash',  
                  layout=Layout(width='auto', height='40px'))

dml_widgets = widgets.VBox([widgets.HBox([dml_left_label, dml_left]), widgets.HBox([dml_right_label, dml_right])])


In [ ]:
statusbar = widgets.HTML("""<style type="text/css">
.cpstatus {background-color: #3C598E;    
        background-image: linear-gradient(#23761f, #79da74);    
        color: white;    
        text-shadow: 3px 3px 5px rgba(0,0,0,0.5); 
        text-align: center;
        font-size: 20px;
        }
</style>
<div class="cpstatus">Find those customers!</div>""")

def statusmessage(s):
    global statusbar
    outstr = """<style type="text/css">
    .cpstatus {background-color: #3C598E;    
            background-image: linear-gradient(#23761f, #79da74);    
            color: white;    
            text-shadow: 3px 3px 5px rgba(0,0,0,0.5); 
            text-align: center;
            font-size: 20px;
            }
    </style>
    <div class="cpstatus">""" + s + """</div>"""
    
    statusbar.value = outstr

In [ ]:
grid = qgrid.show_grid(pd.DataFrame(), show_toolbar=True, grid_options={'forceFitColumns':False, 'enableColumnReorder':True})

grid2 = qgrid.show_grid(pd.DataFrame(), show_toolbar=True, grid_options={'forceFitColumns':False, 'enableColumnReorder':True})

In [ ]:
mf_prog = widgets.IntProgress(
            value = 0,
            max = 10,
            step = 1,
            description = 'Finding gatherings: ',
            bar_style='',
            orientation = 'horizontal', layout=Layout(margin='0px 0px 0px 25px', width='auto'))

meeting_label = widgets.HTML("<h4>Define your criteria for gathering points between customers</h4>")

findmeetings = Button(description=' Find gatherings',
                  button_style='success',
                  icon='users',
                  tooltip='Click to find gatherings of customers using your chosen parameters', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

mtd_label = widgets.HTML("<b>Input the minimum time, in seconds, between events</b>")

m_timedelta = widgets.IntText(layout=Layout(width='auto', height='40px'))

msize_label = widgets.HTML("<b>Approximate size of the outer bounds for a 'gathering'</b>")

m_size = widgets.Dropdown(options=['<100m', 'Building', 'Block', 'District', 'City'])

mentities_label = widgets.HTML("<b>Minimum number of customers involved for a 'gathering'</b>")

min_entities = widgets.IntSlider(description='', 
                  layout=Layout(width='auto', height='40px'), 
                               min=1, 
                               max=10)

displaymeetings = Button(description=' Display tables of gathering data',
                  button_style='success',
                  icon='users',
                  tooltip='Click to display tables of gathering data', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 20px 5px'))

meeting_box = widgets.VBox([meeting_label, mtd_label, m_timedelta, msize_label, m_size,
                            mentities_label, min_entities, findmeetings])

In [ ]:
chartbox = widgets.Output(layout=widgets.Layout(width='auto', height='auto'))

update_chart = widgets.Button(description=' Update Table Heatchart',
                  button_style='success',
                  icon='th-list',
                  tooltip='Click to update the heatchart based on the currently displayed/filtered data table.', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

In [ ]:
plotsearch_label = widgets.HTML("<b>Plot locations from customer interests database</b>")

searchlayer_name = widgets.Text(layout=Layout(width='auto', margin='20px 0px 10px 0px'), 
                    placeholder="Enter a name for your customer interest locations layer.", continuous_update = True)

plotsearch_source = widgets.Text(layout=Layout(width='auto', margin='20px 0px 10px 0px'), 
                    placeholder='Paste in customer interest db id', continuous_update = True)

sm_color_label = widgets.HTML("Select a color for your interest location markers.")

searchmarker_colors = widgets.Select(layout=Layout(width='auto', margin='20px 0px 10px 0px'), options = ['red',
                                                                                                        'darkred',
                                                                                                         'lightred',
                                                                                                         'yellow',
                                                                                                         'orange',
                                                                                                        'blue',
                                                                                                        'darkblue',
                                                                                                        'lightblue',
                                                                                                        'darkgreen',
                                                                                                        'green',
                                                                                                        'lightgreen'
                                                                                                        'purple',
                                                                                                        'black'])

plot_searches = widgets.Button(description=' Plot Interest Locations',
                  button_style='success',
                  icon='search',
                  tooltip='Click to plot locations from customer interests database', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

In [ ]:
searchplot_box = widgets.VBox([plotsearch_label, searchlayer_name, sm_color_label, searchmarker_colors, plotsearch_source, plot_searches])  

In [ ]:
ste_label = widgets.HTML('Treat ID #s as separate customers?')
sel_to_entity = widgets.Checkbox(value=False, disabled=False)

In [ ]:
plot_hmwt = widgets.Button(description=' Plot Day/Hour HM Animation',
                  button_style='success',
                  icon='film',
                  tooltip='Click to create an animated heatmap of customer activity by day of the week and hour of the day.', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

In [ ]:
animations_w = widgets.Select(options=[], layout=widgets.Layout(width='auto'))

plot_locanimation = widgets.Button(description=' Create Activity Animation',
                  button_style='success',
                  icon='film',
                  tooltip='Click to create an animation of customer activity based on your currently-filtered table.', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

In [ ]:
anim_dict = {'hours':'H', 'minutes':'M', 'seconds':'S'}

delete_animation = widgets.Button(description=' Delete the Animation',
                  button_style='success',
                  icon='exit',
                  tooltip='Click to delete the current animation from your map.', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

anim_dur_label = widgets.HTML("Amount of time paths remain on map in animation")

anim_dur_val = widgets.IntText(layout=widgets.Layout(width='auto'))

anim_per_label = widgets.HTML("Amount of time per frame in animation")

anim_per_val = widgets.IntText(layout=widgets.Layout(width='auto'))

anim_dur_units = widgets.Dropdown(options = [i for i in anim_dict], layout=widgets.Layout(width='auto'))

anim_per_units = widgets.Dropdown(options = [i for i in anim_dict], layout=widgets.Layout(width='auto'))

In [ ]:
mbutton = widgets.Button(description=' Re-Render the Master Map',
                  button_style='success',
                  icon='map',
                  tooltip='Click to Render the Master Map', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

hbutton = widgets.Button(description=' Plot Heatmap(s)',
                  button_style='success',
                  icon='fire',
                  tooltip='Click to Plot Heatmap(s) Based on Filtered Table', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

gbutton = widgets.Button(description=' Plot Location(s)',
                  button_style='success',
                  icon='globe',
                  tooltip='Click to Plot Locations(s) Based on Selected Rows', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 5px 5px'))

show_summary = widgets.Checkbox(description="Show Summary?")

In [ ]:
tiles = (leaf.basemap_to_tiles(leaf.basemaps.OpenStreetMap.Mapnik, "2017-04-08"), )
mapbox = leaf.Map(layers=tiles, scroll_wheel_zoom=True, center=(37,-98), zoom=4)

mapbox.layout.height='730px'
mapbox.layout.width='auto'

dc = leaf.DrawControl()

dc.rectangle = {'shapeOptions' : {'color' : 'blue'}}

mapbox.add_control(dc)

mapbox.add_control(leaf.MeasureControl())

mapbox.add_control(leaf.LayersControl(position='topright'))


refreshqueries2 = Button(description=' Refresh Datasets',
                  button_style='success',
                  icon='retweet',
                  tooltip='Click to refresh your list of available datasets', 
                  layout=Layout(width='auto', height='40px', margin='5px 5px 20px 5px'))

summaryw.layout.grid_area = 'sidebar'

mapbox.layout.grid_area = 'main'

topbox2 = widgets.VBox([ql_label, querylist], layout=Layout(width='auto', grid_area='header'))
leftbox = widgets.VBox([refreshqueries, dxwf, entity, widgets.HBox([ste_label, sel_to_entity]), importdx, app_dataset],
                       layout=Layout(width='auto', grid_area='main'))
tab1box = widgets.VBox([topbox2, leftbox, summaryw], layout=Layout(width='auto'))

table_box = widgets.VBox([widgets.HBox([filteron_bb, restore_table]), cm_name, cm_save, summaryw, update_chart, chartbox])

ppics_label = widgets.HTML("<b>Select or enter dataset with geotagged photos</b>")

plotpics_box = widgets.VBox([ppics_label, querylist, refreshqueries2, dxwf, plot_pictures])

animations_box = widgets.VBox([anim_dur_label, widgets.HBox([anim_dur_val, anim_dur_units]),
                               anim_per_label, widgets.HBox([anim_per_val, anim_per_units]),
                               plot_locanimation, plot_hmwt, delete_animation])

custom_layerbox = widgets.Accordion(children=[plotpics_box, searchplot_box, meeting_box, animations_box])

custom_layerbox.set_title(0, 'Plot Geotagged Photos')
custom_layerbox.set_title(1, 'Plot Locations from Interests DB')
custom_layerbox.set_title(2, 'Find and Plot Customer Gatherings')
custom_layerbox.set_title(3, 'Create and Plot Animated Layers')

last_tabbox = widgets.VBox([custom_layerbox, summaryw])

tab2_functions = widgets.Accordion(children=[widgets.VBox([widgets.HBox([dml_label, dml_w]), dml_widgets, mbutton]),
                                             widgets.VBox([label_mastermap, filename_mastermap, save_mastermap]),
                                             widgets.VBox([gh_info_label, gh_text, gh_info])])

tab2_functions.set_title(0, 'Dual Map Layers')
tab2_functions.set_title(1, 'Save The Map as an HTML File')
tab2_functions.set_title(2, 'Reverse Geo Lookup by Geohash')

map_functions = widgets.VBox([tab2_functions, summaryw])

tab = widgets.Tab()

children = [tab1box, map_functions, table_box, custom_layerbox]

tab_titles = ['Import Data','Map Options', 'Table Options', 'Custom Layers']

tab.children = children

for i in range(len(tab_titles)):
    tab.set_title(i, tab_titles[i])
    
tab.layout.grid_area = 'sidebar'
tab.layout.width = 'auto'
    
gb2 = widgets.GridBox(children=[mapbox, tab],
                      layout = Layout(width='auto',
                                      grid_template_rows='auto',
                                      grid_template_columns = '68% 32%',
                                      grid_template_areas = "'main sidebar'"))

cpbanner = widgets.Image(value=open('Dugout_banner.jpg', 'rb').read())
topbox = widgets.VBox([cpbanner, statusbar])

grid.layout.grid_area = 'main'
grid2.layout.grid_area = 'sidebar'


layer_widgets = widgets.HBox([hbutton, save_hmlayer, savelayer_name, gbutton, save_loclayer],
                             layout=Layout(display='flex', align_items='stretch', justify_content='space-between'))
for i in layer_widgets.children:
    i.layout.width = '18%'

gb3 = widgets.GridBox(children=[grid, grid2], layout = Layout(width='auto',
                                                              grid_template_rows='auto',
                                                              grid_template_columns = '68% 32%',
                                                              grid_template_areas = "'main sidebar'"))

grid1_label = widgets.HTML("<h4>Main Data Table</h4>")
grid2_label = widgets.HTML("<h4>Geohash Summary Table</h4>")

grid1_label.layout.grid_area = 'main'
grid2_label.layout.grid_area = 'sidebar'

gb5 = widgets.GridBox(children=[grid1_label, grid2_label],
                      layout = Layout(width='auto',
                                      grid_template_rows='auto',
                                      grid_template_columns = '68% 32%',
                                      grid_template_areas = "'main sidebar'"))

guibox = widgets.VBox([topbox, gb2, layer_widgets, statusbar, gb5, gb3])

In [ ]:
import IPython
IPython.core.display.display(HTML("<style>.container{ width:100% !important; },</style>"))

In [ ]:
clear_output()

display(guibox)
statusmessage("Loading map...")
    
statusmessage("Find those customers!")